In [2]:
pip install graphene

  Using cached graphene-3.3-py2.py3-none-any.whl (128 kB)
  Using cached graphql_core-3.2.3-py3-none-any.whl (202 kB)
  Using cached aniso8601-9.0.1-py2.py3-none-any.whl (52 kB)
  Using cached graphql_relay-3.2.0-py3-none-any.whl (16 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import graphene
from graphene import ObjectType, String, Int, Float, List, Field, DateTime

In [1]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Float, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
from datetime import datetime

Base = declarative_base()

class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String(255))  # Specify length for String
    email = Column(String(255), unique=True)  # Specify length for String
    orders = relationship("Order", backref="user")

class Product(Base):
    __tablename__ = 'products'
    id = Column(Integer, primary_key=True)
    name = Column(String(255))  # Specify length for String
    price = Column(Float)
    description = Column(String(1024))  # Specify length for String

class Order(Base):
    __tablename__ = 'orders'
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.id'))
    product_id = Column(Integer, ForeignKey('products.id'))
    quantity = Column(Integer)
    order_date = Column(DateTime, default=datetime.utcnow)

engine = create_engine('sqlite:///:memory:')
Base.metadata.create_all(engine)

session = sessionmaker(bind=engine)
session = Session()

In [2]:
session.rollback()
# Add sample users
users = [
    User(name="Isak", email="Isak@example.com"),
    User(name="Karthik", email="Karthik@example.com"),
    User(name="Shen", email="Shen@example.com")
]

# Add sample products
products = [
    Product(name="Unicorn_poop", price=9.99, description="Rainbow cupcake"),
    Product(name="Dragon_egg", price=8.99, description="Chocolate"),
    Product(name="Rabbit_leg", price=7.99, description="Cheese cake")
]

session.add_all(users + products)
session.commit()

# Add sample orders
orders = [
    Order(user_id=1, product_id=1, quantity=1),
    Order(user_id=1, product_id=3, quantity=2),
    Order(user_id=2, product_id=2, quantity=1)
]

session.add_all(orders)
session.commit()

In [3]:
for user in session.query(User).all():
    print(f"User: {user.name}, Email: {user.email}")
    for order in user.orders:
        product = session.query(Product).get(order.product_id)
        print(f"    Order: {product.name}, Quantity: {order.quantity}, Date: {order.order_date}")

User: Isak, Email: Isak@example.com
    Order: Unicorn_poop, Quantity: 1, Date: 2023-12-16 01:30:10.638922
    Order: Rabbit_leg, Quantity: 2, Date: 2023-12-16 01:30:10.639119
User: Karthik, Email: Karthik@example.com
    Order: Dragon_egg, Quantity: 1, Date: 2023-12-16 01:30:10.639174
User: Shen, Email: Shen@example.com


In [6]:
class UserType(ObjectType):
    id = Int()
    name = String()
    email = String()
    orders = List(lambda: OrderType)

    def resolve_orders(user, info):
        return session.query(Order).filter(Order.user_id == user.id).all()

class ProductType(ObjectType):
    id = Int()
    name = String()
    price = Float()
    description = String()

class OrderType(ObjectType):
    id = Int()
    user = Field(UserType)
    product = Field(ProductType)
    quantity = Int()
    order_date = DateTime()

    def resolve_user(order, info):
        return session.query(User).filter(User.id == order.user_id).first()

    def resolve_product(order, info):
        return session.query(Product).filter(Product.id == order.product_id).first()

class Query(ObjectType):
    users = List(UserType)
    products = List(ProductType)
    orders = List(OrderType)

    def resolve_users(self, info):
        return session.query(User).all()

    def resolve_products(self, info):
        return session.query(Product).all()

    def resolve_orders(self, info):
        return session.query(Order).all()

schema = graphene.Schema(query=Query)


In [7]:
query_string = '''
{
    users {
        name
        email
    }
}
'''
result = schema.execute(query_string)
print(result.data)

{'users': [{'name': 'Isak', 'email': 'Isak@example.com'}, {'name': 'Karthik', 'email': 'Karthik@example.com'}, {'name': 'Shen', 'email': 'Shen@example.com'}]}
